In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.neighbors import KNeighborsClassifier



from sklearn.model_selection import RandomizedSearchCV
warnings.filterwarnings('ignore')

In [9]:
data = pd.read_csv('../../Data/our_data.csv')
X = data.drop('Class', axis=1)
y = data['Class']

df_for_validators = pd.read_csv('../../Data/validator_data.csv')
X_for_validators = df_for_validators.drop('Class', axis=1)
y_for_validators = df_for_validators['Class']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, stratify=y_val, test_size=0.3, random_state=42
)

X_train = X_train.drop(['Compactness','EquivDiameter', 'Area'], axis=1)
X_val = X_val.drop(['Compactness','EquivDiameter','Area'], axis=1)
X_test = X_test.drop(['Compactness','EquivDiameter','Area'], axis=1)
X_for_validators = X_for_validators.drop(['Compactness','EquivDiameter','Area'], axis=1)
cols = X_train.columns




scaling = sklearn.preprocessing.PowerTransformer(method='box-cox')
X_train = scaling.fit_transform(X_train)
X_test = scaling.transform(X_test)
X_val = scaling.transform(X_val)
X_for_validators = scaling.transform(X_for_validators)

X_train = pd.DataFrame(X_train, columns=cols)
X_test = pd.DataFrame(X_test, columns=cols)
X_val = pd.DataFrame(X_val, columns=cols)
X_for_validators = pd.DataFrame(X_for_validators, columns=cols)

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train.to_frame())
y_encoded = pd.DataFrame(enc.transform(y_train.to_frame()).toarray(),columns=enc.get_feature_names_out(['Class']))
y_val_encoded = pd.DataFrame(enc.transform(y_val.to_frame()).toarray(),columns=enc.get_feature_names_out(['Class']))
y_test_encoded = pd.DataFrame(enc.transform(y_test.to_frame()).toarray(),columns=enc.get_feature_names_out(['Class']))
y_for_validators_encoded = pd.DataFrame(enc.transform(y_for_validators.to_frame()).toarray(),columns=enc.get_feature_names_out(['Class']))

#standard encoding 0,1,2,...
labelencoder = sklearn.preprocessing.LabelEncoder()
y_encoded2 = pd.DataFrame(labelencoder.fit_transform(y_train))
y_val_encoded2 = pd.DataFrame(labelencoder.fit_transform(y_val))
y_test_encoded2 = pd.DataFrame( labelencoder.fit_transform(y_test))
y_for_validators_encoded2 = pd.DataFrame(labelencoder.fit_transform(y_for_validators))


class_names = ['BARBUNYA', 'BOMBAY', 'CALI', 'DERMASON', 'HOROZ', 'SEKER', 'SIRA']

In [10]:
y_encoded

,Class_BARBUNYA,Class_BOMBAY,Class_CALI,Class_DERMASON,Class_HOROZ,Class_SEKER,Class_SIRA
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
7616,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7617,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7618,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7619,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
y_encoded2

,0
0,6
1,5
2,3
3,4
4,4
...,...
7616,6
7617,5
7618,0
7619,3


In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from pyod.models.knn import KNN


def map_gambling(df):
    """
    This function maps the 'CAT_GAMBLING' column in the dataframe from string values to integer values.
    'No' is mapped to 0, 'Low' is mapped to 1, and 'High' is mapped to 2.
    """
    df['CAT_GAMBLING'] = df['CAT_GAMBLING'].map({'No': 0, 'Low': 1, 'High': 2})
    return df


def split_x_from_y(df, target='DEFAULT'):
    """
    This function splits the dataframe into features (X) and target (y) based on the target column name provided.
    """
    X = df.drop(columns=[target])
    y = df[target]
    return X, y


def remove_columns_with_high_correlation(df):
    columns_to_remove = ["T_CLOTHING_12", "T_ENTERTAINMENT_12", "T_GROCERIES_12", "T_GROCERIES_6", "T_HEALTH_12",
                         "T_TAX_12", "T_TAX_6",
                         "T_TRAVEL_12", "T_TRAVEL_6", "T_UTILITIES_12", "T_UTILITIES_6", "T_EXPENDITURE_12",
                         "T_EXPENDITURE_6"]
    df = df.drop(columns=columns_to_remove)
    return df


def automatic_remove_outliers(df):
    clf = KNN(contamination=0.04)
    clf.fit(df)
    df['outliers'] = clf.labels_
    df['outliers'] = df['outliers'].map({'No': 0, 'Yes': 1})
    df = df[df['outliers'] == 0]
    return df


def get_my_metrics(model, X_train, X_val, y_train, y_val):
    """
    This function fits the model with the training data and makes predictions on the validation data.
    It then calculates and returns the accuracy, precision, recall, and f1 score of the model.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    return accuracy, precision, recall, f1


def cox_box_transform(X_train, X_val):
    """
    This function applies the Yeo-Johnson power transformation to the training and validation data.
    """
    cox_box_model = PowerTransformer(method='yeo-johnson')
    X_train_transformed = cox_box_model.fit_transform(X_train)
    X_val_transformed = cox_box_model.transform(X_val)

    # X_train_transformed = scaler.fit_transform(X_train_transformed)
    # X_val_transformed = scaler.transform(X_val_transformed)

    return X_train_transformed, X_val_transformed


def create_tally(X_train, X_val, y_train, y_val, models, names):
    """
    This function performs cross-validation on multiple models and returns a dataframe with the mean and standard deviation of accuracy, precision, recall, and f1 score for each model.
    """
    X = pd.concat([X_train, X_val])
    y = pd.concat([y_train, y_val])
    # if CAT_GAMBLING column has strings, map them to integers
    if X['CAT_GAMBLING'].dtype == 'object':
        X = map_gambling(X)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    stats = pd.DataFrame(
        columns=['model', 'accuracy', 'accuracy_std', 'precision', 'precision_std', 'recall', 'recall_std', 'f1',
                 'f1_std'])

    for model, name in zip(models, names):
        accuracy = []
        precision = []
        recall = []
        f1 = []
        for train_index, val_index in skf.split(X, y):
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]

            X_train_transformed, X_val_transformed = cox_box_transform(X_train, X_val)
            model.fit(X_train_transformed, y_train)
            y_pred = model.predict(X_val_transformed)

            accuracy.append(accuracy_score(y_val, y_pred))
            precision.append(precision_score(y_val, y_pred))
            recall.append(recall_score(y_val, y_pred))
            f1.append(f1_score(y_val, y_pred))

        new_row = {'model': name,
                   'accuracy': np.mean(accuracy),
                   'accuracy_std': np.std(accuracy),
                   'precision': np.mean(precision),
                   'precision_std': np.std(precision),
                   'recall': np.mean(recall),
                   'recall_std': np.std(recall),
                   'f1': np.mean(f1),
                   'f1_std': np.std(f1)}
        stats = pd.concat([stats, pd.DataFrame([new_row])])

    return stats


def validate(X_train, X_test, y_train, y_test, models, names):
    """
    This function fits each model with the training data, makes predictions on the test data, and calculates the accuracy, precision, recall, and f1 score.
    It returns a dataframe with these metrics for each model.
    """
    score = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'recall', 'f1'])


    X_train, X_test = cox_box_transform(X_train, X_test)
    for m, n in zip(models, names):
        ac, pr, re, f1 = get_my_metrics(m, X_train, X_test, y_train, y_test)
        new_row = {'model': n, 'accuracy': ac, 'precision': pr, 'recall': re, 'f1': f1}
        score = pd.concat([score, pd.DataFrame(new_row, index=[0])])
    return score


def prepare_score(score):
    """
    This function reshapes the score dataframe from wide format to long format for visualization.
    """
    score = pd.melt(score, id_vars=['model'], value_vars=['accuracy', 'precision', 'recall', 'f1'])
    return score


def create_visualization(score):
    """
    This function creates a bar plot of the model performance metrics.
    """
    s = prepare_score(score)
    sns.set(rc={'figure.figsize': (10, 5)})
    sns.barplot(x='model', y='value', hue='variable', data=s)
    plt.title('Model Performance')
    plt.ylabel('Score in each metric')
    plt.xlabel('Model')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Metrics')
    plt.show()


In [16]:
# create list of model with random state 42
# create logistic regression with l1 penalty and c=100
# create svm with rbf kernel and c=1000 and gamma=0.1
#create random forest with n_estimators=200 and max_depth=25, min_samples_split=5, criterion='log_loss'
# create decision tree with min_samples_split=10, criterion='entropy', max_depth=20
models=[LogisticRegression(penalty='l2', C=100, random_state=42),
        SVC(kernel='rbf', C=1000, gamma=0.1, random_state=42),
        RandomForestClassifier(n_estimators=200, max_depth=25, min_samples_split=5, criterion='gini', random_state=42),
        DecisionTreeClassifier(min_samples_split=10, criterion='entropy', max_depth=20, random_state=42)]

names=['Logistic Regression', 'SVM', 'Random Forest', 'Decision Tree']

score=validate(X_train, X_for_validators, y_train, y_for_validators_encoded['Class_BARBUNYA'], models, names)
score

ValueError: Mix of label input types (string and number)